In [12]:
from google.colab import files

# This will prompt you to select and upload the ZIP file
uploaded = files.upload()


Saving Smart-Waste-Classification-System-main.zip to Smart-Waste-Classification-System-main.zip


In [14]:
import zipfile
import os

# This should match the filename you uploaded
zip_file_path = 'Smart-Waste-Classification-System-main.zip'
extract_folder = 'Smart-Waste-Classification-System'

# Create the extract folder if it doesn’t exist
os.makedirs(extract_folder, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print("✅ Extraction completed! Files are in:", extract_folder)


✅ Extraction completed! Files are in: Smart-Waste-Classification-System


In [38]:
import cv2
import os

img_size = (224, 224)
dataset_path = "/content/Waste"

count = 0

for category in os.listdir(dataset_path):
    category_path = os.path.join(dataset_path, category)
    if os.path.isdir(category_path):
        for img_name in os.listdir(category_path):
            img_path = os.path.join(category_path, img_name)
            if os.path.isfile(img_path) and img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                try:
                    img = cv2.imread(img_path)
                    if img is not None:
                        img = cv2.resize(img, img_size)
                        cv2.imwrite(img_path, img)
                        count += 1
                    else:
                        print(f"Could not read image: {img_path}")
                except Exception as e:
                    print(f"Error processing {img_path}: {e}")
    else:
        print(f"Skipping non-directory: {category_path}")

print(f"Resized {count} images to {img_size}")


Resized 0 images to (224, 224)


In [54]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load MobileNetV2 as feature extractor (no top layers)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze base model layers so they won't be trained
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output_layer = Dense(6, activation='softmax')(x)  # 6 classes

model = Model(inputs=base_model.input, outputs=output_layer)

# Compile the model before saving
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Save the model using the recommended Keras native format
model.save('waste_classifier_mobile.keras')

print("Model saved successfully!")


Model saved successfully!


In [57]:
pip install tensorflow opencv-python numpy


In [2]:
# Step 1: Import libraries
from google.colab import files
import tensorflow as tf
import numpy as np
import cv2
from PIL import Image
import io

# Step 2: Load the trained model
model = tf.keras.models.load_model("waste_classifier_mobile.keras")
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 3: Define class labels (must match your training order)
labels = ['Cardboard', 'E waste', 'Glass', 'Metal', 'Paper', 'Plastic']

# Step 4: Upload image
print("📤 Please upload a waste image (jpg, png, etc.)")
uploaded = files.upload()  # 👈 THIS LINE

# Step 5: Load and preprocess the uploaded image
for filename in uploaded.keys():
    try:
        img = Image.open(io.BytesIO(uploaded[filename])).convert("RGB")
    except Exception as e:
        print(f"❌ Error opening image: {e}")
        continue

    img = img.resize((128, 128))
    img_array = np.array(img).astype("float32") / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Step 6: Make prediction
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    confidence = np.max(prediction)

    # Step 7: Show result
    print(f"✅ Predicted Class: {labels[predicted_class]}")
    print(f"🔍 Confidence: {confidence:.2f}")


📤 Please upload a waste image (jpg, png, etc.)


Saving plastic bottle.png to plastic bottle.png


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
✅ Predicted Class: Metal
🔍 Confidence: 0.45


In [5]:
from google.colab import files
import tensorflow as tf
import numpy as np
import cv2
from PIL import Image
import io

# Upload image
uploaded = files.upload()

# Load model
model = tf.keras.models.load_model("waste_classifier_mobile.keras")

# Define labels
labels = ['Cardboard', 'E waste', 'Glass', 'Metal', 'Paper', 'Plastic']

# Load and preprocess image
for filename in uploaded.keys():
    img_bytes = uploaded[filename]
    img = Image.open(io.BytesIO(img_bytes)).convert("RGB")
    img = img.resize((128, 128))
    img_array = np.array(img).astype("float32") / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    confidence = np.max(prediction)

    print(f"Predicted: {labels[predicted_class]} ({confidence:.2f})")


Saving metal can.jpg to metal can.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted: Metal (0.29)
